In [115]:
import pandas as pd
from sklearn import datasets, linear_model, metrics
import numpy as np
from sklearn.metrics import mean_squared_error
import tensorflow_decision_forests as tfdf

  Obtaining dependency information for tensorflow_decision_forests from https://files.pythonhosted.org/packages/6e/68/ecfe92f30e74ec385bd3dabf23b61e623cc14b9a63add037d93afa5d449f/tensorflow_decision_forests-1.9.0-cp311-cp311-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for tensorflow~=2.16.1 from https://files.pythonhosted.org/packages/6d/69/9999c2d9e8a3b08dfcfc7e9259a05fb1da5f700936091d2eb4a7985c2776/tensorflow-2.16.2-cp311-cp311-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for tf-keras~=2.16 from https://files.pythonhosted.org/packages/8a/ed/e08afca471299b04a34cd548e64e89d0153eda0e6cf9b715356777e24774/tf_keras-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for tf-keras~=2.16 from https://files.pythonhosted.org/packages/21/8b/75f7572ec0273ed8da50bc19defe08aaaafcc15fda3407db53f49acec814/tf_keras-2.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for tf-keras~=2.16 from https://files.pythonhosted.org/packages/75/aa/cf09f8956d4f276f655b13674e15d8d6015fd832f9689aa9ff2a515781ab/tf_keras-2.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for rich from https://files.pythonhosted.org/packages/19/71/39c7c0d87f8d4e6c020a393182060eaefeeae6c01dab6a84ec346f2567df/rich-13.9.4-py3-none-any.whl.metadata
  Obtaining dependency information for namex from https://files.pythonhosted.org/packages/73/59/7854fbfb59f8ae35483ce93493708be5942ebb6328cd85b3a609df629736/namex-0.0.8-py3-none-any.whl.metadata
  Obtaining dependency information for optree from

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 619.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.6/589.6 kB 337.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 383.2 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0


2024-12-29 14:04:07.003589: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [117]:
train = pd.read_csv('../data/processed/train_data_one_hot.csv')
test = pd.read_csv('../data/processed/test_data_one_hot.csv')

In [118]:
#split input and output
X_train=train.drop(['SalePrice'], axis=1)
y_train= train['SalePrice']
X_test=test.drop(['SalePrice'], axis=1)
y_test= test['SalePrice']

In [119]:
#Linear regression: 
#fit_intercept=False is needed as collinearity of one hot encoding, first row has to be dropped. 
reg = linear_model.LinearRegression(fit_intercept=False)
reg.fit(X_train, y_train)


LinearRegression(fit_intercept=False)

In [120]:
#predict sales price on test set
predictions = reg.predict(X_test)

In [121]:
#Apply inverse transformation to bring predictions back to the original scale
predictions = np.exp(predictions)

In [122]:
#RMSE 
rms = mean_squared_error(np.exp(y_test), predictions, squared=False)
rms

767926.2644360319

In [ ]:
#tf decision trees
#!pip install tensorflow_decision_forests

In [123]:
#load main file as one hot encoding not needed
train=pd.read_csv('../data/processed/train_data_reg.csv')
test=pd.read_csv('../data/processed/test_data_reg.csv')

In [137]:
# Convert the pandas dataframe into a TensorFlow dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="SalePrice", task = tfdf.keras.Task.REGRESSION)

In [138]:
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)
model.compile


Use /var/folders/lj/1fkt9p5950q_xh36l9svgyyw0000gn/T/tmpqksoww9l as temporary training directory


<bound method InferenceCoreModel.compile of <tensorflow_decision_forests.keras.RandomForestModel object at 0x14aa2f590>>

In [139]:
model.fit(train_ds)

Reading training dataset...
Training dataset read in 0:00:00.972009. Found 1165 examples.
Training model...


[INFO 24-12-29 14:28:18.9297 GMT kernel.cc:1233] Loading model from path /var/folders/lj/1fkt9p5950q_xh36l9svgyyw0000gn/T/tmpqksoww9l/model/ with prefix ac357a053efe4a73


Model trained in 0:00:02.101136
Compiling model...


[INFO 24-12-29 14:28:19.2181 GMT decision_forest.cc:734] Model loaded with 300 root(s), 111084 node(s), and 74 input feature(s).
[INFO 24-12-29 14:28:19.2182 GMT abstract_model.cc:1344] Engine "RandomForestGeneric" built
[INFO 24-12-29 14:28:19.2182 GMT kernel.cc:1061] Use fast generic engine


Model compiled.


In [140]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="SalePrice", task = tfdf.keras.Task.REGRESSION)


In [141]:
model.compile(metrics=["accuracy"])
print(model.evaluate(test_ds))

1/1 [==============================] - 1s 609ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
[0.0, 0.0]


In [143]:
inspector = model.make_inspector()
inspector.evaluation()

Evaluation(num_examples=1165, accuracy=None, loss=None, rmse=0.15414591857731405, ndcg=None, aucs=None, auuc=None, qini=None)

In [ ]:
#note this RMSE is on the logarmithic data, not the actual data! 

In [144]:
#calculate the actual RMSE
preds = model.predict(test_ds)

1/1 [==============================] - 0s 214ms/step


In [145]:
predics=np.exp(preds)
y=test['SalePrice']

In [148]:
rms = mean_squared_error(np.exp(y), preds, squared=False)
rms

192198.77392215197

In [149]:

#next acitivity, explore all of these methods and compare RMSE of all

tfdf.keras.get_all_models()


[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]